<a href="https://colab.research.google.com/github/Khoan-IT/TrainingWithTPU/blob/main/CustomData_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np, pandas as pd, os
import tensorflow as tf, re, math
import sys

In [ ]:
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
GCS_OUT = "gs://data_train_cifar/cifar"
GCS_PATTERN = GCS_OUT + "*.tfrec"
SHARD = 25
SHARD_SIZE = 2000 
TARGET_SIZE = [32,32]
# BATCH_SIZE = 16*strategy.num_replicas_in_sync  # A TPU has 8 cores so this will be 128
BATCH_SIZE = 512
VALIDATION_SPLIT = 0.2

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(feature0, feature1):
  feature = {
      'image': _bytes_feature(feature0),
      'label': _int64_feature(feature1)
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
for shard in range(SHARD):
  #print('Writing TFRecord %i'%(shard))
  filename = GCS_OUT + "{:02d}-{}.tfrec".format(shard, SHARD_SIZE)
  with tf.io.TFRecordWriter(filename) as writer:
    for shard_size in range(SHARD_SIZE):
      img = cv2.cvtColor(x_train[shard*shard_size], cv2.COLOR_RGB2BGR)
      img = cv2.imencode('.jpg',img,(cv2.IMWRITE_JPEG_QUALITY,94))[1].tobytes()
      #name = "image_{}".format(shard*shard_size)
      label = y_train[shard*shard_size][0]
      example = serialize_example(img,label)
      writer.write(example)
print("Write Successful!")

TRAINING MODEL RESNET50 WITH TPU USE DATASET CIFAR100


In [ ]:
try: # detect TPUs
  tpu = None
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
  strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
  #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: grpc://10.36.61.154:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.36.61.154:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of accelerators:  8


In [ ]:
def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string) # shape [] means scalar
        "label": tf.io.FixedLenFeature([], tf.int64)
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    
    # FixedLenFeature fields are now ready to use: exmple['size']
    # VarLenFeature fields require additional sparse_to_dense decoding
    
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.reshape(image, [*TARGET_SIZE, 3])
    label = example['label']
    one_hot_class = tf.one_hot(label,100)

    return image, one_hot_class

In [ ]:
# splitting data files between training and validation
filenames = tf.io.gfile.glob(GCS_PATTERN)
split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]
print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(len(filenames), len(training_filenames), len(validation_filenames)))
validation_steps = int(50000 // len(filenames) * len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(50000 // len(filenames) * len(training_filenames)) // BATCH_SIZE
print("With a batch size of {}, there will be {} batches per training epoch and {} batch(es) per validation run.".format(BATCH_SIZE, steps_per_epoch, validation_steps))

Pattern matches 25 data files. Splitting dataset into 20 training files and 5 validation files
With a batch size of 512, there will be 78 batches per training epoch and 19 batch(es) per validation run.


In [ ]:
def load_dataset(filenames):
  # read from TFRecords. For optimal performance, read from multiple
  # TFRecord files at once and set the option experimental_deterministic = False
  # to allow order-altering optimizations.

  option_no_order = tf.data.Options()
  option_no_order.experimental_deterministic = False

  dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
  dataset = dataset.with_options(option_no_order)
  dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
  return dataset

In [ ]:
def get_batched_dataset(filenames, train=False):
  dataset = load_dataset(filenames)
  dataset = dataset.cache() # This dataset fits in RAM
  if train:
    # Best practices for Keras:
    # Training dataset: repeat then batch
    # Evaluation dataset: do not repeat
    dataset = dataset.repeat()
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
  # should shuffle too but this dataset was well shuffled on disk already
  return dataset
  # source: Dataset performance guide: https://www.tensorflow.org/guide/performance/datasets

# instantiate the datasets
training_dataset = get_batched_dataset(training_filenames, train=True)
validation_dataset = get_batched_dataset(validation_filenames, train=False)

In [ ]:
with strategy.scope(): # this line is all that is needed to run on TPU (or multi-GPU, ...)
  model = tf.keras.applications.ResNet50(weights=None, input_shape=[*TARGET_SIZE, 3], classes=100)
  model.compile(
    optimizer='adam',
    steps_per_execution = 50,
    loss='categorical_crossentropy',
    metrics=['accuracy'])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
from time import process_time, time
EPOCHS = 50
t1_start = time()  
history = model.fit(training_dataset, 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=EPOCHS,
                    validation_data=validation_dataset,
                    validation_steps =validation_steps,  
                    verbose=True)
t1_stop = time()    
print("Elapsed time during the whole program in seconds model", t1_stop-t1_start) 

Epoch 1/50
78/78 [==============================] - 68s 872ms/step - loss: 4.6980 - accuracy: 0.0847 - val_loss: 4.4866 - val_accuracy: 0.0167
Epoch 2/50
78/78 [==============================] - 2s 22ms/step - loss: 3.2876 - accuracy: 0.2732 - val_loss: 103.1300 - val_accuracy: 0.0148
Epoch 3/50
78/78 [==============================] - 2s 22ms/step - loss: 2.8925 - accuracy: 0.3388 - val_loss: 3.7390 - val_accuracy: 0.2597
Epoch 4/50
78/78 [==============================] - 2s 22ms/step - loss: 2.0605 - accuracy: 0.5250 - val_loss: 4.2923 - val_accuracy: 0.2756
Epoch 5/50
78/78 [==============================] - 2s 22ms/step - loss: 2.0443 - accuracy: 0.5335 - val_loss: 3.0924 - val_accuracy: 0.4089
Epoch 6/50
78/78 [==============================] - 2s 22ms/step - loss: 1.3567 - accuracy: 0.6818 - val_loss: 2.4585 - val_accuracy: 0.4667
Epoch 7/50
78/78 [==============================] - 2s 22ms/step - loss: 1.0633 - accuracy: 0.7457 - val_loss: 2.1501 - val_accuracy: 0.5454
Epoch 8/5